In [61]:
import random
import pandas as pd
import numpy as np
import nltk
import PyPDF2

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge

In [8]:
# Import the data to a df
df = pd.read_csv('data/siop_ml_train_participant.csv')

# Confirm that the data has been imported and is formatted correctly
df.head(3)

,Respondent_ID,open_ended_1,open_ended_2,open_ended_3,open_ended_4,open_ended_5,E_Scale_score,A_Scale_score,O_Scale_score,C_Scale_score,N_Scale_score
0,10446116527,"I would change my vacation week, because I am ...",I would reach out to my boss and ask him or he...,I would not go. I am a not a social person. I ...,I would ask my manager why he/she gave me such...,I would find this experience super enjoyable. ...,2.250000,3.750000,3.166667,3.750000,2.916667
1,10440100535,I would talk to my colleague and see if they w...,I would continue to work on the project that w...,I would talk to my colleague and try to talk t...,I would feel upset about the negative feedback...,I would find this experience enjoyable. I feel...,4.666667,4.416667,4.583333,5.000000,1.333333
2,10462850071,I would feel upset because perhaps I already b...,I would start working on the project now and g...,I would feel guilty about thinking about not g...,I would feel really defensive about it. I woul...,I would find it enjoyable because I would be r...,2.250000,4.750000,4.083333,4.666667,2.166667


In [20]:
train = df
print(train[1:6])

   Respondent_ID                                       open_ended_1  \
1    10440100535  I would talk to my colleague and see if they w...   
2    10462850071  I would feel upset because perhaps I already b...   
3    10460008027  I would suggest that whoever requested the tim...   
4    10459746373  I would talk to my colleague to see if he has ...   
5    10460002321  It would depend on how serious my plans were. ...   

                                        open_ended_2  \
1  I would continue to work on the project that w...   
2  I would start working on the project now and g...   
3  I would try to finish early because it's alway...   
4  I would remind my boss that I am working on th...   
5  I would speak with my boss and let him know th...   

                                        open_ended_3  \
1  I would talk to my colleague and try to talk t...   
2  I would feel guilty about thinking about not g...   
3  I wouldn't want to go, but I'd have to weigh t...   
4  I would g

In [100]:
words = train["open_ended_1"][1].split()
print(nltk.pos_tag(words))


[('I', 'PRP'), ('would', 'MD'), ('talk', 'VB'), ('to', 'TO'), ('my', 'PRP$'), ('colleague', 'NN'), ('and', 'CC'), ('see', 'VB'), ('if', 'IN'), ('they', 'PRP'), ('were', 'VBD'), ('willing', 'JJ'), ('to', 'TO'), ('change', 'VB'), ('the', 'DT'), ('vacation', 'NN'), ('date', 'NN'), ('because', 'IN'), ('I', 'PRP'), ('really', 'RB'), ('need', 'VBP'), ('to', 'TO'), ('have', 'VB'), ('this', 'DT'), ('week', 'NN'), ('off.', 'RB'), ('If', 'IN'), ('the', 'DT'), ('colleague', 'NN'), ('was', 'VBD'), ('not', 'RB'), ('willing', 'JJ'), ('to', 'TO'), ('change', 'VB'), ('it', 'PRP'), ('I', 'PRP'), ('would', 'MD'), ('go', 'VB'), ('talk', 'NN'), ('to', 'TO'), ('my', 'PRP$'), ('boss.', 'NN'), ('I', 'PRP'), ('would', 'MD'), ('explain', 'VB'), ('to', 'TO'), ('my', 'PRP$'), ('boss', 'NN'), ('that', 'IN'), ('I', 'PRP'), ('would', 'MD'), ('still', 'RB'), ('like', 'VB'), ('to', 'TO'), ('have', 'VB'), ('the', 'DT'), ('week', 'NN'), ('off', 'IN'), ('I', 'PRP'), ('requested', 'VBD'), ('but', 'CC'), ('if', 'IN'), ('i

In [105]:
def presentTag(variable):
    present = 0
    words = variable.split()
    #print(nltk.pos_tag(words))
    for word in nltk.pos_tag(words):
        #print(word)
        if word == "VBG" or "VBP" or "VPZ":
            present += 1
            print (word)
            print(present)
            #print("here")
        else:
            present = present
            print("not here")
        percentpresent = present/len(variable.split())
        print(percentpresent)
        return percentpresent
    
train["present"] = train["open_ended_1"].apply(presentTag)

# Let's check our shape here:
print(train.shape)

# Let's also view a specific, random respondent.  If they are 
print (train.iloc[np.random.randint(0, len(train))])

PRP
1
0.019230769230769232
PRP
1
0.009900990099009901
PRP
1
0.013513513513513514
PRP
1
0.022222222222222223
PRP
1
0.022222222222222223
PRP
1
0.017857142857142856
PRP
1
0.023809523809523808
PRP
1
0.02631578947368421
IN
1
0.02631578947368421
PRP
1
0.01818181818181818
PRP
1
0.012987012987012988
PRP
1
0.01818181818181818
PRP
1
0.009900990099009901
PRP
1
0.0625
NN
1
0.02564102564102564
PRP
1
0.030303030303030304
VBG
1
0.01639344262295082
PRP
1
0.011904761904761904
PRP
1
0.018867924528301886
PRP
1
0.022222222222222223
PRP
1
0.01098901098901099
PRP
1
0.012658227848101266
PRP
1
0.01694915254237288
PRP
1
0.011111111111111112
PRP
1
0.014492753623188406
PRP
1
0.015384615384615385
PRP
1
0.023809523809523808
NN
1
0.024390243902439025
RB
1
0.04
PRP
1
0.022727272727272728
PRP
1
0.015625
PRP
1
0.015873015873015872
IN
1
0.04
PRP
1
0.047619047619047616
IN
1
0.01639344262295082
PRP
1
0.016129032258064516
PRP
1
0.037037037037037035
PRP
1
0.01639344262295082
PRP
1
0.029411764705882353
PRP
1
0.0099009900990

PRP
1
0.027777777777777776
PRP
1
0.0070921985815602835
PRP
1
0.03125
PRP
1
0.02857142857142857
PRP
1
0.02
PRP
1
0.02631578947368421
PRP
1
0.027777777777777776
PRP
1
0.011627906976744186
PRP
1
0.02
PRP
1
0.019230769230769232
PRP
1
0.03571428571428571
PRP
1
0.009900990099009901
IN
1
0.015384615384615385
PRP
1
0.02702702702702703
PRP
1
0.03125
PRP
1
0.017543859649122806
PRP
1
0.023809523809523808
NN
1
0.0196078431372549
PRP
1
0.015873015873015872
PRP
1
0.018518518518518517
PRP
1
0.01818181818181818
PRP
1
0.014925373134328358
PRP
1
0.011627906976744186
PRP
1
0.0196078431372549
NNP
1
0.018518518518518517
IN
1
0.023809523809523808
PRP
1
0.02127659574468085
IN
1
0.018867924528301886
PRP
1
0.01694915254237288
PRP
1
0.05
PRP
1
0.022727272727272728
PRP
1
0.022727272727272728
PRP
1
0.018518518518518517
PRP
1
0.015625
PRP
1
0.02857142857142857
PRP
1
0.038461538461538464
PRP
1
0.02702702702702703
IN
1
0.025
PRP
1
0.01694915254237288
PRP
1
0.029411764705882353
PRP
1
0.037037037037037035
PRP
1
0.0384

PRP
1
0.017241379310344827
PRP
1
0.03333333333333333
PRP
1
0.02127659574468085
PRP
1
0.014084507042253521
PRP
1
0.01818181818181818
PRP
1
0.023809523809523808
PRP
1
0.021739130434782608
PRP
1
0.02564102564102564
IN
1
0.05
PRP
1
0.01818181818181818
PRP
1
0.029411764705882353
PRP
1
0.023255813953488372
PRP
1
0.014925373134328358
PRP
1
0.008928571428571428
PRP
1
0.025
PRP
1
0.02857142857142857
PRP
1
0.023255813953488372
PRP
1
0.017857142857142856
PRP
1
0.02564102564102564
PRP
1
0.02040816326530612
PRP
1
0.024390243902439025
PRP
1
0.03333333333333333
RB
1
0.029411764705882353
PRP
1
0.017543859649122806
PRP
1
0.009523809523809525
PRP
1
0.011904761904761904
PRP
1
0.02702702702702703
PRP
1
0.01818181818181818
PRP
1
0.024390243902439025
PRP
1
0.022727272727272728
PRP
1
0.030303030303030304
DT
1
0.02
IN
1
0.020833333333333332
PRP
1
0.01694915254237288
IN
1
0.022222222222222223
PRP
1
0.016666666666666666
PRP
1
0.023255813953488372
PRP
1
0.02127659574468085
PRP
1
0.022222222222222223
IN
1
0.03125

PRP
1
0.024390243902439025
PRP
1
0.029411764705882353
PRP
1
0.037037037037037035
PRP
1
0.05555555555555555
PRP
1
0.0125
PRP
1
0.034482758620689655
PRP
1
0.037037037037037035
PRP
1
0.02857142857142857
PRP
1
0.016129032258064516
PRP
1
0.06666666666666667
PRP
1
0.02702702702702703
PRP
1
0.022222222222222223
PRP
1
0.0078125
PRP
1
0.020833333333333332
(1088, 16)
Respondent_ID                                          10440100545
open_ended_1     I would most likely change my vacation plans u...
open_ended_2     I would work on the project as long as I have ...
open_ended_3     I would still go to the networking meeting wit...
open_ended_4     I would make sure that I developed a sound and...
open_ended_5     I would find learning about Norway to be enjoy...
E_Scale_score                                              4.33333
A_Scale_score                                                 4.25
O_Scale_score                                              4.58333
C_Scale_score                        

In [109]:
def firstpersonTag(variable):
    firstperson = 0
    #print(variable.split())
    for word in variable.split():
        if word == "I" or "me" or "my" or "mine" or "Me" or "My" or "Mine":
            print(word)
            firstperson += 1
            print(firstperson)
        else:
            firstperson = firstperson
            print(word)
            print(firstperson)
        percentfirst = firstperson/len(variable.split())
        return percentfirst

train["fstpersn"] = train["open_ended_1"].apply(firstpersonTag)

# Let's check our shape here:
print(train.shape)

# Let's also view a specific, random respondent.  If they are 
print (train.iloc[np.random.randint(0, len(train))])

I
1
I
1
I
1
I
1
I
1
It
1
I
1
I
1
If
1
I
1
I
1
I
1
I
1
I
1
i
1
I
1
Assuming
1
I
1
I
1
I
1
I
1
I
1
I
1
I
1
It
1
I
1
I
1
i
1
Well
1
I
1
I
1
I
1
If
1
I
1
If
1
I
1
I
1
I
1
I
1
I
1
I
1
First,
1
I
1
It
1
I
1
As
1
I
1
I
1
I
1
I
1
I
1
I
1
I
1
I
1
I
1
I
1
I
1
I
1
I
1
It
1
I'd
1
I
1
I
1
I
1
I
1
I
1
I
1
I
1
If
1
I
1
I
1
I
1
I
1
I
1
I
1
I
1
I
1
I
1
I
1
I
1
I
1
If
1
I
1
I
1
I
1
I
1
I
1
I
1
I
1
I
1
I
1
I
1
I
1
It
1
What
1
Since
1
I
1
I
1
I
1
I
1
I
1
I
1
I
1
I
1
I
1
First,
1
I
1
I
1
I
1
If
1
I
1
I
1
I
1
I
1
I
1
I
1
I
1
I
1
I
1
I
1
I
1
I
1
I
1
I
1
I
1
I
1
I
1
If
1
I
1
If
1
I
1
I
1
I
1
I
1
I
1
I
1
I
1
I
1
I
1
I
1
I
1
I
1
I
1
I
1
I
1
I
1
I
1
I
1
I
1
I
1
I'd
1
I
1
I
1
I
1
I
1
I
1
It
1
if
1
I
1
I
1
I
1
I
1
I
1
I
1
I
1
I
1
I
1
I
1
I
1
I
1
I
1
I'd
1
I
1
I
1
I
1
I
1
I
1
I
1
I
1
I
1
I
1
I
1
It
1
I
1
I
1
I
1
I
1
I
1
I
1
Unless
1
I
1
I
1
I
1
I
1
I
1
I
1
I
1
If
1
It
1
I
1
I
1
I
1
I
1
I
1
Unless
1
I
1
I
1
I
1
I
1
I
1
I
1
I
1
I
1
I
1
Before
1
I
1
I
1
I
1
I
1
I
1
I
1
It
1
I
1
I
1
I
1
I
1
If
1
I
1
I
1
I
1
I
1
I
1
I
1

In [62]:
file = open('LM_Negative.pdf', 'rb')

# creating a pdf reader object
fileReader = PyPDF2.PdfFileReader(file)
pageObj = fileReader.getPage(0)
page_content = pageObj.extractText()
#print(page_content)

In [63]:
negativewords = ""
for page in range(fileReader.numPages):
    pageObj = fileReader.getPage(page)
    page_content = pageObj.extractText()
    negativewords = negativewords + page_content
#print(negativewords)
negwords = negativewords.split()

negwords = negwords[7:]

In [92]:
def negAffect(variable):
    
    words = variable.split()
    #print(words)
    for word in words:
        print(word)
        negative = 0
        if word.upper() in negwords:
            negative = negative + 1
        print(negative)
    return negative
            #percentnegative = negative/len(words)
            #return percentnegative
        

In [94]:
train["negative"] = train["open_ended_4"].apply(negAffect)

# Let's check our shape here:
print(train.shape)

# Let's also view a specific, random respondent.  If they are 
print (train.iloc[np.random.randint(0, len(train))])

I
0
would
0
ask
0
my
0
manager
0
why
0
he/she
0
gave
0
me
0
such
0
feedback.
0
But
0
although
0
I
0
may
0
not
0
agree
0
with
0
it
0
and
0
not
0
believe
0
it's
0
true,
0
maybe
0
I
0
am
0
in
0
the
0
wrong.
0
I
0
would
0
ask
0
my
0
manager
0
what
0
I
0
needed
0
to
0
do
0
to
0
correct
0
the
0
wrong.
0
If
0
everything
0
is
0
BS
0
though,
0
and
0
I
0
hate
0
the
0
job,
0
I
0
may
0
just
0
quit.
0
I
0
would
0
feel
0
upset
1
about
0
the
0
negative
1
feedback.
0
I
0
would
0
approach
0
my
0
boss
0
and
0
ask
0
him/her
0
why
0
I
0
was
0
given
0
negative
1
feedback
0
and
0
how
0
I
0
thought
0
it
0
was
0
inaccurate.
0
I
0
would
0
let
0
my
0
boss
0
know
0
that
0
I
0
don't
0
want
0
this
0
to
0
effect
0
my
0
work.
0
I
0
would
0
ask
0
if
0
it
0
was
0
possible
0
that
0
he/she
0
did
0
not
0
give
0
the
0
right
0
feedback.
0
I
0
would
0
feel
0
really
0
defensive
1
about
0
it.
0
I
0
would
0
confront
1
my
0
manager
0
and
0
tell
0
them
0
my
0
side
0
of
0
the
0
story.
0
Before
0
I
0
did
0
that
0
though
0
I
0
woul

1
feedback
0
laying
0
down.
0
I
0
would
0
probably
0
feel
0
upset.
0
I
0
would
0
try
0
my
0
best
0
to
0
not
0
say
0
anything
0
to
0
lose
1
my
0
bonus.
0
If
0
I
0
did
0
feel
0
the
0
need
0
to
0
say
0
something
0
I
0
would
0
say
0
it
0
very
0
calmly.
0
I
0
would
0
act
0
like
0
it
0
is
0
no
0
big
0
deal,
0
and
0
forget
0
about
0
it.
0
instead
0
i
0
would
0
try
0
to
0
do
0
a
0
lot
0
better,
0
and
0
have
0
everyone
0
who
0
is
0
important
0
and
0
accessible
0
to
0
notice
0
my
0
efforts.
0
in
0
that
0
way,
0
it
0
will
0
be
0
obvious
0
that
0
the
0
manager's
0
assessment
0
was
0
in
0
correct.
0
Well
0
I
0
would
0
talk
0
to
0
my
0
boss
0
and
0
let
0
them
0
know
0
that
0
this
0
feedback
0
is
0
false.
0
If
0
they
0
didn't
0
let
0
up
0
I
0
would
0
go
0
to
0
human
0
resources.
0
If
0
that
0
doesn't
0
work
0
I
0
would
0
call
0
the
0
news
0
channel.
0
I
0
would
0
feel
0
pisaed
0
because
0
they
0
believed
0
it.
0
I
0
would
0
sit
0
down
0
and
0
have
0
a
0
conversation
0
with
0
my
0
boss.
0
I
0
would
0


0
in
0
his
0
assessment.
0
If
0
I
0
deserved
0
the
0
criticism,
0
I
0
will
0
take
0
it
0
to
0
heart
0
and
0
try
0
to
0
correct
0
it.
0
I
0
would
0
bring
0
the
0
feedback
0
to
0
my
0
manager
0
and
0
ask
0
them
0
to
0
explain
0
further
0
what
0
I
0
did
0
wrong
1
and
0
how
0
I
0
can
0
improve
0
in
0
the
0
future.
0
I
0
would
0
also
0
try
0
to
0
see
0
if
0
I
0
could
0
find
0
a
0
way
0
around
0
it
0
and
0
possibly
0
have
0
the
0
feedback
0
reversed.
0
Or
0
I
0
would
0
see
0
if
0
it
0
could
0
be
0
reversed
0
pending
0
good
0
behavior
0
and
0
no
0
negative
1
feedback
0
in
0
the
0
future.
0
I
0
would
0
feel
0
like
0
my
0
manager
0
was
0
treating
0
me
0
unfairly.
0
I
0
would
0
first
0
go
0
to
0
my
0
manager
0
to
0
discuss
0
the
0
feedback.
0
If
0
this
0
did
0
not
0
got
0
well
0
I
0
would
0
escalate
1
to
0
someone
0
above
0
the
0
manager
0
to
0
discuss
0
my
0
work
0
performance.
0
I
0
would
0
feel
0
aghast
0
that
0
someone
0
would
0
question
1
my
0
work
0
and
0
I
0
would
0
wonder
0
what
0
they
0

0
never
0
happened
0
again.
0
fighting
0
with
0
your
0
boss
0
is
0
never
0
wise.
0
I
0
would
0
ask
0
my
0
manager
0
to
0
show
0
me
0
some
0
proof
0
of
0
the
0
negative
1
feedback
0
they
0
gave
0
me.
0
I
0
would
0
also
0
voice
0
the
0
importance
0
of
0
that
0
want,
0
and
0
that
0
I
0
believe
0
I
0
am
0
a
0
good
0
and
0
valuable
0
worker
0
and
0
deserve
0
my
0
annual
0
bonus.
0
If
0
they
0
have
0
proof
0
to
0
show
0
me
0
and
0
back
0
up
0
their
0
opinion
0
then
0
lets
0
discuss
0
further.
0
I
0
would
0
explain
0
that
0
although
0
I
0
disagree
1
with
0
the
0
criticism
1
I
0
would
0
take
0
it
0
in
0
consideration.
0
I
0
would
0
evaluate
0
the
0
feedback
0
to
0
see
0
if
0
there
0
is
0
any
0
necessary
0
changes
0
that
0
I
0
needed
0
to
0
make.
0
My
0
employment
0
at
0
the
0
company
0
is
0
very
0
important
0
because
0
this
0
is
0
how
0
I
0
provide
0
for
0
my
0
family,
0
so
0
therefore
0
I
0
would
0
view
0
the
0
feedback
0
as
0
a
0
positive
0
and
0
not
0
view
0
it
0
as
0
negative.
0
I
0
would


0
the
0
situation,
0
I
0
may
0
stick
0
up
0
for
0
myself
0
if
0
need
0
be,
0
and
0
try
0
to
0
compromise
0
with
0
my
0
boss.
0
However,
0
if
0
it
0
is
0
something
0
minor,
0
I
0
might
0
just
0
mention
0
that
0
that
0
was
0
not
0
my
0
intention,
0
I
0
wasn't
0
aware
0
that
0
I
0
was
0
performing
0
in
0
that
0
way,
0
suck
0
it
0
up,
0
and
0
move
0
on.
0
I
0
would
0
be
0
upset
1
about
0
the
0
situation
0
because
0
I
0
would
0
feel
0
that
0
I
0
had
0
been
0
treated
0
unfairly.
0
If
0
I
0
truly
0
felt
0
that
0
the
0
negative
1
feedback
0
would
0
affect
0
my
0
pay
0
or
0
position,
0
I
0
would
0
go
0
to
0
my
0
manager
0
and
0
ask
0
him
0
or
0
her
0
to
0
explain
0
to
0
me
0
why
0
he/she
0
felt
0
that
0
I
0
deserved
0
that
0
feedback.
0
If
0
I
0
still
0
felt
0
it
0
wasn't
0
true,
0
I
0
would
0
explain
0
why
0
I
0
felt
0
I
0
didn't
0
deserve
0
it.
0
If
0
I
0
was
0
satisfied
0
with
0
his/her
0
explanation
0
and
0
thought
0
I
0
did
0
deserve
0
the
0
negative
1
feedback,
0
I
0
would
0
ask
0
for
0
h

...

my
0
support
0
as
0
to
0
why
0
I
0
don't
0
agree
0
with
0
the
0
feedback.
0
Lastly,
0
I
0
would
0
follow
0
procedure
0
with
0
contesting
1
the
0
negative
1
feedback.
0
I
0
could
0
feel
0
very
0
bad
1
but
0
I
0
will
0
try
0
to
0
talk
0
to
0
my
0
manager
0
to
0
understand
0
if
0
this
0
is
0
a
0
personal
0
problem.
0
I
0
think
0
that
0
money
0
is
0
always
0
important
0
but
0
that
0
is
0
not
0
why
0
I
0
accept
0
injustice.
0
I
0
would
0
feel
0
very
0
upset.
0
I
0
may
0
try
0
to
0
talk
0
to
0
my
0
manager
0
and
0
figure
0
out
0
why
0
the
0
negative
1
feedback.
0
At
0
the
0
same
0
time
0
try
0
to
0
do
0
my
0
job
0
better
0
so
0
leave
0
no
0
chance
0
for
0
a
0
negative
1
feedback.
0
I
0
would
0
try
0
to
0
reason
0
and
0
explain
0
myself.
0
Then
0
maybe
0
ask
0
for
0
a
0
redo
0
of
0
the
0
project
0
how
0
my
0
boss
0
would
0
like
0
it.
0
Then
0
if
0
that's
0
not
0
the
0
option.
0
I
0
would
0
go
0
to
0
HR
0
or
0
whoever
0
ranks
0
above
0
my
0
manager.
0
I
0
would
0
more
0
than
0
likely
0
feel
0


0
and
0
not
0
let
0
it
0
undermine
1
me.
0
I
0
would
0
not
0
outwardly
0
address
0
these
0
false
1
allegations
1
but
0
appear
0
that
0
I
0
accept
0
the
0
feedback
0
and
0
will
0
do
0
better.
0
I
0
would
0
put
0
forth
0
consistent
0
and
0
good
0
work
0
to
0
overcome
1
this.
0
I
0
would
0
feel
0
horrible
0
about
0
this
0
situation
0
and
0
incredibly
0
stressed.
0
I
0
would
0
ask
0
for
0
a
0
meeting
0
with
0
my
0
manager
0
to
0
go
0
over
0
the
0
feedback
0
and
0
discuss
0
how
0
I
0
can
0
strive
0
to
0
do
0
better.
0
I
0
would
0
talk
0
to
0
the
0
boss
0
about
0
what
0
they
0
saw
0
and
0
how
0
they
0
determined
0
the
0
results.
0
If
0
I
0
still
0
disagreed
1
I
0
would
0
go
0
to
0
their
0
boss
0
and
0
request
0
another
0
review
0
or
0
an
0
appeal
0
of
0
the
0
review
0
expressing
0
my
0
case.
0
I
0
would
0
talk
0
with
0
my
0
manager.
0
I
0
would
0
try
0
to
0
get
0
their
0
vantage
0
point.
0
If
0
that
0
didn't
0
work,
0
I
0
would
0
try
0
to
0
talk
0
to
0
someone
0
above
0
my
0
manager.
0
I
0
w

supervisor
0
and
0
ask
0
for
0
constructive
0
criticism
1
so
0
I
0
can
0
be
0
a
0
better
0
employee.
0
I
0
would
0
voice
0
my
0
opinions
0
where
0
I
0
think
0
I
0
was
0
right,
0
but
0
I
0
would
0
learn
0
from
0
this
0
mistake
1
and
0
be
0
better
0
next
0
time.
0
I
0
would
0
feel
0
pretty
0
nervous
0
and
0
upset
1
with
0
myself
0
with
0
negative
1
feedback
0
that
0
could
0
carry
0
real
0
consequences.
0
I
0
would
0
ask
0
my
0
manager
0
for
0
a
0
brief
0
meeting
0
to
0
discuss
0
their
0
report
0
and
0
defend
1
myself.
0
I
0
would
0
also
0
ask
0
the
0
standing
0
of
0
my
0
bonus.
0
I
0
would
0
feel
0
upset
1
that
0
my
0
manager
0
felt
0
this
0
way.
0
I
0
would
0
really
0
look
0
into
0
the
0
negative
1
things
0
he
0
said
0
and
0
see
0
if
0
there
0
was
0
something
0
I
0
was
0
missing.
0
I
0
would
0
work
0
on
0
those
0
things
0
to
0
improve
0
them
0
for
0
next
0
time.
0
I
0
don't
0
think
0
he
0
was
0
doing
0
it
0
on
0
purpose.
0
I
0
would
0
be
0
very
0
upset,
0
but
0
he
0
is
0
the
0
manager.


0
voice
0
my
0
concerns
1
with
0
my
0
manager
0
and
0
tell
0
his
0
or
0
her
0
that
0
I
0
don’t
0
agree
0
with
0
the
0
feedback.
0
Then
0
I
0
would
0
ask
0
my
0
colleagues
0
for
0
advice.
0
Finally,
0
I
0
would
0
find
0
out
0
ways
0
to
0
avoid
0
any
0
consequences
0
by
0
going
0
through
0
company
0
channels
0
with
0
other
0
managers.
0
I
0
would
0
complain
1
to
0
the
0
board
0
of
0
directors
0
that
0
I
0
was
0
graded
0
harshly
1
and
0
unfairly.
0
I
0
would
0
try
0
to
0
explain
0
to
0
my
0
boss
0
that
0
the
0
feedback
0
was
0
not
0
fairly
0
justified
0
and
0
that
0
there
0
should
0
be
0
no
0
real
0
consequences.
0
I
0
would
0
also
0
try
0
to
0
make
0
him
0
pay
0
for
0
his
0
consequences
0
as
0
he
0
falsely
1
gave
0
me
0
negative
1
feedback.
0
I
0
would
0
feel
0
somewhat
0
negatively
1
about
0
the
0
feedback
0
but
0
I
0
would
0
do
0
my
0
best
0
to
0
fix
0
whatever
0
what
0
wrong
1
and
0
discuss
0
with
0
my
0
supervisor
0
how
0
I
0
could
0
do
0
better
0
and
0
how
0
I
0
am
0
driven
0
to
0
a

and
0
how
0
i
0
can
0
fix
0
it.
0
I
0
would
0
want
0
to
0
show
0
i
0
care
0
and
0
can
0
be
0
better
0
I
0
would
0
speak
0
with
0
my
0
manager
0
and
0
explain
0
to
0
him
0
in
0
a
0
polite
0
manner
0
why
0
I
0
don't
0
agree
0
with
0
his
0
feedback.
0
I
0
should
0
not
0
lose
1
a
0
bonus
0
for
0
something
0
negative.
0
I
0
would
0
take
0
the
0
negative
1
feedback
0
and
0
turn
0
it
0
into
0
a
0
positive
0
thing
0
if
0
he's
0
willing
0
to
0
work
0
with
0
me.
0
I
0
would
0
research
0
the
0
facts
0
and
0
confront
1
him
0
with
0
it..and
0
say
0
why
0
I
0
don't
0
agree.
0
I
0
might
0
also
0
get
0
support
0
from
0
others
0
on
0
this,
0
and
0
see
0
how
0
they
0
were
0
reviewed.
0
I
0
would
0
dispute
1
the
0
feedback.
0
I
0
would
0
try
0
to
0
find.flaws
0
in
0
his
0
thinking
0
and
0
maybe
0
find
0
proof
0
of
0
his
0
errors.
0
If
0
I
0
didn't
0
agree
0
and
0
felt
0
he
0
was
0
acting
0
unfairly
1
I
0
would
0
go
0
over
0
his
0
head.
0
I
0
would
0
take
0
time
0
to
0
appraise
0
the
0
feedback
0
carefull